In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import practice_library as ice
from random import *

##
top = 1
bottom = 2
left = 3
right = 4
##

width = 6
length = 5

startpoint = [2, 3] 
startpoint_place = top

endpoint = [3]
endpoint_place = right

##

startpoint_maps = np.zeros((len(startpoint) , 2))
endpoint_maps = np.zeros((len(endpoint), 2))

# set startpoint
if startpoint_place == top:
    startpoint_maps[:, 0] = 0
    startpoint_maps[:, 1] = np.transpose(startpoint)
    
elif startpoint_place == bottom:
    startpoint_maps[:, 0] = length + 1
    startpoint_maps[:, 1] = np.transpose(startpoint)
    
elif startpoint_place == left:
    startpoint_maps[:, 0] = np.transpose(startpoint)
    startpoint_maps[:, 1] = 0
    
elif startpoint_place == right:
    startpoint_maps[:, 0] = np.transpose(startpoint)
    startpoint_maps[:, 1] = width + 1
    
#set endpoint

if endpoint_place == top:
    endpoint_maps[:, 0] = 0
    endpoint_maps[:, 1] = np.transpose(endpoint)
    
elif endpoint_place == bottom:
    endpoint_maps[:, 0] = length + 1
    endpoint_maps[:, 1] = np.transpose(endpoint)
    
elif startpoint_place == left:
    endpoint_maps[:, 0] = np.transpose(endpoint)
    endpoint_maps[:, 1] = 0
    
elif endpoint_place == right:
    endpoint_maps[:, 0] = np.transpose(endpoint)
    endpoint_maps[:, 1] = width + 1
    
    
#
startrand = randint(1, np.shape(startpoint)[0]) - 1
startpoint = startpoint_maps[startrand, :]
startpoint = startpoint.astype(int)
startpoint_maps = startpoint_maps.astype(int)
endpoint_maps = endpoint_maps.astype(int)
maps = maps.astype(int)
#

maps = np.zeros((length + 2, width + 2))
maps[0, :] = 1
maps[length + 1, :] = 1
maps[:, 0] = 1
maps[:, width + 1] = 1

for i in range(0, startpoint_maps.shape[0]):
    maps[startpoint_maps[i][0], startpoint_maps[i - 1][1]] = 2

for i in range(0, endpoint_maps.shape[0]):
    maps[endpoint_maps[i][0], endpoint_maps[i - 1][1]] = 2

maps[startpoint[0], startpoint[1]] = 3





In [ ]:
def nextile(now, maps, horizontal):
    #restart value
    re_next = 0
       
    #load line
    if horizontal == 1:
        line = maps[now[0], :]
        now_line = now[1]
        
    elif horizontal == 0:
        line = np.transpose(maps[:, now[1]])
        now_line = now[0]
        
    #adjust line array
    for i in range(0, line.shape[0]):
        if i < now_line:
            line[i] = line[i] * -1
        elif i == now_line:
            line[i] = 0
        # if i > now_line:
        # line [i] = line[i] * 1
        
    #last nega/posi num
    for i in range(0, line.shape[0]):
        if line[i] < 0:
            leftend = i
            
    for i in range(line.shape[0] - 1, -1, -1)
        if line[i] > 0:
            rightend = i
            
    #randomly decide next tile
    while re_next = 1:
        next_tile = randint(leftend + 1, rightend - 1)
        if next_tile == now_line:
            re_next = 1
        elif next_tile != now_line:
            re_next = 0
            
    if horizontal == 1:
        now[1] = next_tile
    elif horizontal == 0:
        now[0] = next_tile
        


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import practice_library as ice
from random import *

width = 6
length = 5

startpoint = [2, 3] 
startpoint_place = 1

endpoint = [3]
endpoint_place = 4

# top = 1
# bottom = 2
# left = 3
# right = 4


##

startpoint_maps = np.zeros((len(startpoint) , 2))
endpoint_maps = np.zeros((len(endpoint), 2))

# set startpoint
if startpoint_place == top:
    startpoint_maps[:, 0] = 0
    startpoint_maps[:, 1] = np.transpose(startpoint)
    horizontal_s = 0
    
elif startpoint_place == bottom:
    startpoint_maps[:, 0] = length + 1
    startpoint_maps[:, 1] = np.transpose(startpoint)
    horizontal_s = 0

elif startpoint_place == left:
    startpoint_maps[:, 0] = np.transpose(startpoint)
    startpoint_maps[:, 1] = 0
    horizontal_s = 1

elif startpoint_place == right:
    startpoint_maps[:, 0] = np.transpose(startpoint)
    startpoint_maps[:, 1] = width + 1
    horizontal_s = 1

#set endpoint

if endpoint_place == top:
    endpoint_maps[:, 0] = 0
    endpoint_maps[:, 1] = np.transpose(endpoint)
    horizontal_e = 0
    
elif endpoint_place == bottom:
    endpoint_maps[:, 0] = length + 1
    endpoint_maps[:, 1] = np.transpose(endpoint)
    horizontal_e = 0

elif startpoint_place == left:
    endpoint_maps[:, 0] = np.transpose(endpoint)
    endpoint_maps[:, 1] = 0
    horizontal_e = 1

elif endpoint_place == right:
    endpoint_maps[:, 0] = np.transpose(endpoint)
    endpoint_maps[:, 1] = width + 1
    horizontal_e = 1

    

#
startrand = randint(1, np.shape(startpoint)[0]) - 1
startpoint = startpoint_maps[startrand, :]
startpoint = startpoint.astype(int)
startpoint_maps = startpoint_maps.astype(int)
endpoint_maps = endpoint_maps.astype(int)
#

maps = np.zeros((length + 2, width + 2))
maps[0, :] = 1
maps[length + 1, :] = 1
maps[:, 0] = 1
maps[:, width + 1] = 1

maps = maps.astype(int)

for i in range(0, startpoint_maps.shape[0]):
    maps[startpoint_maps[i][0], startpoint_maps[i - 1][1]] = 2

for i in range(0, endpoint_maps.shape[0]):
    maps[endpoint_maps[i][0], endpoint_maps[i - 1][1]] = 2

maps[startpoint[0], startpoint[1]] = 3

## start
now = startpoint
horizontal = horizontal_s


print("startpoint", startpoint)
print("now", now)
print("maps")
print(maps)
##
now, maps, horizontal = ice.nextile(now, maps, horizontal)




In [8]:
import numpy as np
import matplotlib.pyplot as plt
import practice_library as ice
from random import *

################################################################
####################### < input > ##############################
width = 6
length = 5

# top = 1, bottom = 2, left = 3, right = 4

startpoint = [2, 3] 
startpoint_place = 1

endpoint = [3]
endpoint_place = 4

################################################################
################################################################

#initial info
startpoint, startpoint_maps, endpoint_maps, horizontal_all = ice.initinfo(startpoint, startpoint_place, endpoint, endpoint_place, width, length)

endpoint = endpoint_maps[0]


#initial horizontal
horizontal_s = horizontal_all[0]
horizontal_e = horizontal_all[1]


#initial maps
maps = np.zeros((length + 2, width + 2))
maps[0, :] = 1
maps[length + 1, :] = 1
maps[:, 0] = 1
maps[:, width + 1] = 1

maps = maps.astype(int)

for i in range(0, startpoint_maps.shape[0]):
    maps[startpoint_maps[i][0], startpoint_maps[i - 1][1]] = 2

for i in range(0, endpoint_maps.shape[0]):
    maps[endpoint_maps[i][0], endpoint_maps[i - 1][1]] = 2

maps[startpoint[0], startpoint[1]] = 3


## start
now = startpoint.copy()
horizontal = horizontal_s

keep = 1
joint = 0

while keep == 1:

    #save now
    now_latest = now.copy()
    maps_latest = maps.copy()

    #next tile
    stuck = 1
    while stuck == 1:
        # not stuck yet
        stuck = 0
        
        #next tile
        now, maps, horizontal = ice.nextile(now, maps, horizontal)

        # see if stuck
        stuck = ice.imstuck(now, maps, horizontal)
        if stuck == 1:
            now = now_latest.copy()
            maps = maps_latest.copy()
    
    print("now", now)

    # swap horizontal
    if horizontal == 0:
        horizontal = 1
    else: #horizontal = 1
        horizontal = 0

    #joint
    joint = joint + 1
    if joint > 5:
        keep = 0
#     #endcheck
#     if joint > 5:
#         end = ice.endcheck(now, maps, endpoint_place, horizontal, horizontal_e, width, length)
#         if end == 1:
#             keep = 0


print(endpoint[0])

print("maps")
print(maps)



now [3 3]
now [3 1]
now [5 1]
now [5 2]
now [3 2]
now [3 4]
3
maps
[[1 1 2 3 1 1 1 1]
 [1 0 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 1]
 [1 0 0 0 0 1 0 2]
 [1 0 0 1 0 0 0 1]
 [1 0 0 1 0 0 0 1]
 [1 1 1 1 1 1 1 1]]
